# This is the workbook for estimating BD

In [222]:
from tensorly.decomposition import tensor_train 
from tensorly import tt_to_tensor
import numpy as np

def get_estimate(input_vector, n_sites, max_bd, thershold):
    for bd in range(1, max_bd + 1):
        
        rank_list = [1]
        for i in range(n_sites - 1):
            rank_list.append(2)
        rank_list.append(1)
        
        mps_factors = tensor_train(input_vector, rank=bd)
        mps_differ = input_vector - tt_to_tensor(mps_factors)
        
        diff_num = np.round(np.sum(np.abs(mps_differ)),7)
        
        print(f"Error at BD {bd}: {diff_num}")
        
        if diff_num < thershold:
            return bd, diff_num
    
    return max_bd, diff_num


In [223]:
from tensorly import random

n_sites = 6
# state_1d = np.random.rand(2 ** n_sites)
state_1d = [0 for i in range(2 ** n_sites)]
state_1d[32] = 1 / np.sqrt(6)  # |100000>
state_1d[16] = 1 / np.sqrt(6)  # |010000>
state_1d[8]  = 1 / np.sqrt(6)  # |001000>
state_1d[4]  = 1 / np.sqrt(6)  # |000100>
state_1d[2]  = 1 / np.sqrt(6)  # |000010>
state_1d[1]  = 1 / np.sqrt(6)  # |000001>
state_1d[3] = 1
state_1d[7] = 1
tensor_shape = (2, ) * n_sites
max_bd = 10
thershold = 1E-100

state_1d = np.array(state_1d)
norm = np.linalg.norm(state_1d)
state_1d = state_1d / norm
print(len(state_1d))
print(f"The norm of the input tensor: {np.linalg.norm(state_1d)}")

64
The norm of the input tensor: 1.0


In [224]:
tensor = np.reshape(state_1d, tensor_shape)
estimated_bd, diff_num = get_estimate(tensor, n_sites, max_bd, thershold)
print(f"Estimated bd = {estimated_bd}")

Error at BD 1: 1.4410985
Error at BD 2: 0.7863948
Error at BD 3: 0.0
Estimated bd = 3


In [225]:
n_sites = 6
state_1d = [0 for i in range(2 ** n_sites)]
state_1d[32] = 1 / np.sqrt(6)  # |100000>
state_1d[16] = 1 / np.sqrt(6)  # |010000>
state_1d[8]  = 1 / np.sqrt(6)  # |001000>
state_1d[4]  = 1 / np.sqrt(6)  # |000100>
state_1d[2]  = 1 / np.sqrt(6)  # |000010>
state_1d[1]  = 1 / np.sqrt(6)  # |000001>

tensor_shape = (2, ) * n_sites
max_bd = 10
thershold = 1E-100

state_1d = np.array(state_1d)
norm = np.linalg.norm(state_1d)
state_1d = state_1d / norm

n_sites = 6
state_1d_add = np.random.rand(2 ** n_sites) ** 0.1

state_1d += state_1d_add
norm = np.linalg.norm(state_1d)
state_1d = state_1d / norm


print(f"The norm of the input tensor: {np.linalg.norm(state_1d)}")
tensor = np.reshape(state_1d, tensor_shape)
estimated_bd, diff_num = get_estimate(tensor, n_sites, max_bd, thershold)
print(estimated_bd)

The norm of the input tensor: 0.9999999999999999
Error at BD 1: 0.7170554
Error at BD 2: 0.5262594
Error at BD 3: 0.3396048
Error at BD 4: 0.1904756
Error at BD 5: 0.1104263
Error at BD 6: 0.0740752
Error at BD 7: 0.0285745
Error at BD 8: 0.0
8


In [214]:
n_sites = 6
state_1d = np.random.rand(2 ** n_sites)
state_1d[7] = 1
tensor_shape = (2, ) * n_sites
max_bd = 10
thershold = 0.001

state_1d = np.array(state_1d)
norm = np.linalg.norm(state_1d)
state_1d = state_1d / norm
print(len(state_1d))

64


In [185]:
tensor = np.reshape(state_1d, tensor_shape)
estimated_bd = get_estimate(tensor, n_sites, max_bd, thershold)
print(f"Estimated bd = {estimated_bd}")

2.9745543
2.2189813
1.5239765
0.8683777
0.5804254
0.2714453
0.0310227
0.0
Estimated bd = 8


# BD estimate of the actual Hamiltonian

In [227]:
import time
import csv
import inspect
import numpy as np
from util_hamil import test,test1,test2,test5
from util_mutualinfo import mutual_all
from util_covar import covariance
from ten_network import mps1,cpd1,mps2,cpd2
from util_save import printx,save_parameters
from util_gfro import (obtain_fragment,
                       rotated_hamiltonian,
                       boson_eigenspectrum_sparse,
                       boson_eigenspectrum_full,
                       quad_diagonalization)



n = 3 # number of modes

# Define Hamiltonian using parameters

h_variables = [1,1,1,0.6,0.6,0.6] # variables goes in to Hamiltonian

truncation = 6  # Occuppation number (Number of basis function)

def extract_function_name(func):
    return func.__name__

hamil_name = extract_function_name(test2)

H = test2(h_variables) # Generate Hamiltonian iterms of OpenFermion Bosonic Operators from "util_hamil.py"
                        #test1 contains only upto quadratic terms
maxit = 1

options = {
        'maxiter' : maxit, # maximum iteration goes to the cost function
        'gtol': 1e-7,  # Set the tolerance for cost function change
 #       'xatol': 1e-7,
        'disp'    : False
    }


start_time = time.time()

## Mutual information and MPS and CP errors before rotation
#eigenvalues1,e1 = boson_eigenspectrum_sparse(H, truncation, 1)
eigenvalues1,e1 = boson_eigenspectrum_full(H, truncation)
e1 = e1[:,0]
f1=e1.reshape(truncation,truncation,truncation)
print(f1.shape)

(6, 6, 6)


In [228]:
estimated_bd = get_estimate(f1, 3, 20, thershold)

Error at BD 1: 0.091342
Error at BD 2: 0.0034609
Error at BD 3: 0.0
